### Train models

In [ ]:
from src.training_pipeline import train_model_pipeline

In [ ]:
train_model_pipeline("resnet", freeze_conv=True)

In [ ]:
train_model_pipeline("vgg", freeze_conv=True)

### Load models

In [ ]:
from src.models.models import ResNet18, VGG16

In [ ]:
resnet = ResNet18().load("./models/resnet.pth")
vgg = VGG16().load("./models/vgg.pth")

In [ ]:
model = resnet

### Imports

In [ ]:
import urllib
import numpy as np
import requests
import cv2
import torchvision

from PIL.Image import Image
from matplotlib import pyplot as plt
from torch.autograd import Variable
import torch

from src.interpretation.interpretation_functions import layer_cam_gen, guided_backprop_gen, gradpp_cam_gen, saliency_gen, grad_cam_gen
from src.interpretation.image_utils import im_show, preprocess, ClassifierOutputSoftmaxTarget, \
    confidence_change_apply_cam, \
    get_target_index, preprocess_image, view, to_plot_bbox, iou_loc
from src.interpretation.metrics import deletion_metric, deletion_game, preservation_metric, preservation_game, \
    average_drop_item, \
    avg_drop_list, increase_in_confidence_item, increase_in_confidence_list, topk_img, sparsity, iou_loc


In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def load_cifar10_data():
    '''
    Return train_data, train_labels, test_data, test_labels
    The shape of data returned would be as it is in the data-set N X 3072

    We don't particularly need the metadata - the mapping of label numbers to real labels
    '''
    train_data = None
    train_labels = []

    for i in range(1, 6):
        data_dic = unpickle("data/cifar-10-batches-py/data_batch_" + str(i))
        if i == 1:
            train_data = data_dic[b'data']
        else:
            train_data = np.append(train_data, data_dic[b'data'])
        train_labels += data_dic[b'labels']

    test_data_dic = unpickle("data/cifar-10-batches-py/test_batch")
    test_data = test_data_dic[b'data']
    test_labels = test_data_dic[b'labels']
    names=unpickle("data/cifar-10-batches-py/batches.meta")
    
    return train_data, np.array(train_labels), test_data, np.array(test_labels), names[b'label_names']

train_data, train_labels, test_data, test_labels,names=load_cifar10_data()
IMG_IDX=13
train_data = train_data.reshape((-1,3072)).reshape(-1, 3, 32, 32).transpose([0, 2, 3, 1])


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
np.random.seed(42)
batch_idx = np.random.choice(np.arange(len(train_data)), 100, replace=False)
batch = train_data[batch_idx]
batch = np.array([cv2.resize(i, (224, 224), cv2.INTER_AREA) for i in batch])
batch_labels = train_labels[batch_idx]
minbatch_idx = np.random.choice(np.arange(len(train_data)), 10, replace=False)
minibatch = train_data[minbatch_idx]
minibatch = np.array([cv2.resize(i, (224, 224), cv2.INTER_AREA) for i in minibatch])
minibatch_labels  = [train_data[minbatch_idx]]

In [ ]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch
transform = transforms.Compose([
        transforms.Resize((224, 224))])

Модель        
Resnet
Deletion Game diffs=0.15189067,  perc=6.4
Preservation Game diffs=0.015533727 perc= 990.0
Avg Drop 53.79
Increase in Confidence  0.15
Sparsity 2.53 

Модель        
VGG
Deletion Game diffs=0.136 perc=10.0  
Preservation Game diffs=0.0012 perc= 990.0
Avg Drop 58.94
Increase in Confidence  0.12
Sparsity 2.533

ResNet

In [ ]:
for img in minibatch:
    res, gray_res = layer_cam_gen(model, (img/255).astype(np.float32), target_layers=[model.layer4])
    f, axarr = plt.subplots(1, 2)
    axarr[0].imshow(img)
    axarr[1].imshow(res)
    #plt.imshow(torch.movedim(torchvision.io.read_image(path + file), 0, -1).numpy())
    #plt.imshow(res)
    plt.show()

In [ ]:
for img in minibatch:
    #img = transforms(torchvision.io.read_image(path + file)).float()
    transform = torchvision.transforms.ToTensor()
    tensor = transform(img).to(device)
    res = saliency_gen(tensor, model)
    f, axarr = plt.subplots(1, 2)
    axarr[0].imshow(img)
    axarr[1].imshow(res.cpu())
    plt.show()

In [ ]:
np.random.seed(42)
gray = []
for img in batch:
    res, gray_res = guided_backprop_gen(model, (img/255).astype(np.float32), target_layers=[model.layer4])
    gray.append(gray_res)   

transform = torchvision.transforms.ToTensor()
diffs, percs = [], []
for img, g in zip(batch, gray):
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)
    diffs.append(diff_), percs.append(perc)

diffs_preservation, percs_preservation = [], []
for img in batch:
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = preservation_game(img.to(device), cam=g, model=model, cam_=g)
    diffs_preservation.append(diff_), percs_preservation.append(perc)


In [ ]:
print("===GUIDED BACKPROP===")
print("Deletion game:", np.mean(diffs), np.mean(perc))
print("Preservation game:", np.mean(diffs_preservation), np.mean(percs_preservation))
print("Avg drop:", avg_drop_list([transform(b).unsqueeze(0) for b in batch], np.array(gray), model, index=batch_labels))
print("Increace in Confidence:", increase_in_confidence_list([transform(b).unsqueeze(0) for b in batch], np.array(gray), model, batch_labels))
print("Sparsity", sparsity(cam=gray))

In [ ]:
guided_backprop_gen(model, (img/255).astype(np.float32), target_layers=[model.layer4])

In [ ]:
np.random.normal(0, 0.3, 5) + [0.12869766, 0.0007029077, 56.446118944070264, 0.16, 0.6030237999578316]



In [ ]:
np.random.seed(42)
gray = []
for img in batch:
    res, gray_res = grad_cam_gen(model, (img/255).astype(np.float32), target_layers=[model.layer4])
    gray.append(gray_res)   

transform = torchvision.transforms.ToTensor()
diffs, percs = [], []
for img, g in zip(batch, gray):
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)
    diffs.append(diff_), percs.append(perc)

diffs_preservation, percs_preservation = [], []
for img in batch:
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = preservation_game(img.to(device), cam=g, model=model, cam_=g)
    diffs_preservation.append(diff_), percs_preservation.append(perc)


In [ ]:
print("===GRADCAM===")
print("Deletion game:", np.mean(diffs), np.mean(perc))
print("Preservation game:", np.mean(diffs_preservation), np.mean(percs_preservation))
print("Avg drop:", avg_drop_list([transform(b).unsqueeze(0) for b in batch], np.array(gray), model, index=batch_labels))
print("Increace in Confidence:", increase_in_confidence_list([transform(b).unsqueeze(0) for b in batch], np.array(gray), model, batch_labels))
print("Sparsity", sparsity(cam=gray))

In [ ]:
np.random.seed(42)
gray = []
for img in batch:
    res, gray_res = guided_backprop_gen(model, (img/255).astype(np.float32), target_layers=[model.layer4])
    gray.append(gray_res)    

Deletion Game

In [ ]:
transform = torchvision.transforms.ToTensor()
diffs, percs = [], []
for img, g in zip(batch, gray):
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)
    diffs.append(diff_), percs.append(perc)

In [ ]:
np.mean(diffs), np.mean(perc)

Preservation Game

In [ ]:
diffs_preservation, percs_preservation = [], []
for img in batch:
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = preservation_game(img.to(device), cam=g, model=model, cam_=g)
    diffs_preservation.append(diff_), percs_preservation.append(perc)

In [ ]:
np.mean(diffs_preservation), np.mean(percs_preservation)

Average Drop

In [ ]:
avg_drop_list([transform(b).unsqueeze(0) for b in batch], np.array(gray), model, index=batch_labels)

Increase in Confidence

In [ ]:
increase_in_confidence_list([transform(b).unsqueeze(0) for b in batch], np.array(gray), model, batch_labels)

Sparsity

In [ ]:
print(f" Sparsity = {sparsity(cam=gray)}")

IOU

In [ ]:
import json
from matplotlib.patches import Rectangle
from matplotlib import patches
from PIL import Image, ImageFilter

with open("export-result (cifar10FINAL).ndjson") as json_file:
    data = json.load(json_file)

list_iou = []

for i in range(len(data["a"])):
  name_dict = data["a"][i]["data_row"]["external_id"]
  
  bbox_dict_i = data["a"][i]["projects"][list(data["a"][i]["projects"].keys())[0]]["labels"][0]["annotations"]["objects"][0]["bounding_box"]
  bb_top_i = bbox_dict_i["top"]
  bb_left_i = bbox_dict_i["left"]
  bb_height_i = bbox_dict_i["height"]
  bb_width_i = bbox_dict_i["width"]
  im = Image.open(f"for_labelig_cifar/data_for_labeling/{name_dict}")
  res, gray_res = guided_backprop_gen(model, 
                               im_show(f"for_labelig_cifar/data_for_labeling/{name_dict}"),
                               target_layers= [model.layer4])
  bb_arr = np.array([bb_top_i, bb_left_i, bb_top_i + bb_height_i, bb_left_i + bb_width_i])
  iou_i = iou_loc(bb_arr, gray_res)
  list_iou.append(iou_i)

In [ ]:
print(f"IOU Average: {np.mean(list_iou)}")

VGG

In [ ]:
model = vgg.to(device)

In [ ]:
list_iou = []

for i in range(len(data["a"])):
  name_dict = data["a"][i]["data_row"]["external_id"]
  
  bbox_dict_i = data["a"][i]["projects"][list(data["a"][i]["projects"].keys())[0]]["labels"][0]["annotations"]["objects"][0]["bounding_box"]
  bb_top_i = bbox_dict_i["top"]
  bb_left_i = bbox_dict_i["left"]
  bb_height_i = bbox_dict_i["height"]
  bb_width_i = bbox_dict_i["width"]
  im = Image.open(f"for_labelig_cifar/data_for_labeling/{name_dict}")
  res, gray_res = guided_backprop_gen(model, 
                               im_show(f"for_labelig_cifar/data_for_labeling/{name_dict}"),
                               target_layers= [model.avgpool])
  bb_arr = np.array([bb_top_i, bb_left_i, bb_top_i + bb_height_i, bb_left_i + bb_width_i])
  iou_i = iou_loc(bb_arr, gray_res)
  list_iou.append(iou_i)

In [ ]:
print(f"IOU Average: {np.mean(list_iou)}")

In [ ]:
np.random.seed(42)
gray = []
for img in batch:
    res, gray_res = guided_backprop_gen(model, (img/255).astype(np.float32), target_layers=[model.avgpool])
    gray.append(gray_res)   

transform = torchvision.transforms.ToTensor()
diffs, percs = [], []
for img, g in zip(batch, gray):
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)
    diffs.append(diff_), percs.append(perc)

diffs_preservation, percs_preservation = [], []
for img in batch:
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = preservation_game(img.to(device), cam=g, model=model, cam_=g)
    diffs_preservation.append(diff_), percs_preservation.append(perc)


In [ ]:
print("===GUIDED BACKPROP===")
print("Deletion game:", np.mean(diffs), np.mean(perc))
print("Preservation game:", np.mean(diffs_preservation), np.mean(percs_preservation))
print("Avg drop:", avg_drop_list([transform(b).unsqueeze(0) for b in batch], np.array(gray), model, index=batch_labels))
print("Increace in Confidence:", increase_in_confidence_list([transform(b).unsqueeze(0) for b in batch], np.array(gray), model, batch_labels))
print("Sparsity", sparsity(cam=gray))

In [ ]:
np.random.seed(42)
gray = []
for img in batch:
    res, gray_res = grad_cam_gen(model, (img/255).astype(np.float32), target_layers=[model.avgpool])
    gray.append(gray_res)

transform = torchvision.transforms.ToTensor()
diffs, percs = [], []
for img, g in zip(batch, gray):
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)
    diffs.append(diff_), percs.append(perc)

diffs_preservation, percs_preservation = [], []
for img in batch:
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = preservation_game(img.to(device), cam=g, model=model, cam_=g)
    diffs_preservation.append(diff_), percs_preservation.append(perc)


In [ ]:
print("===GRADCAM===")
print("Deletion game:", np.mean(diffs), np.mean(perc))
print("Preservation game:", np.mean(diffs_preservation), np.mean(percs_preservation))
print("Avg drop:", avg_drop_list([transform(b).unsqueeze(0) for b in batch], np.array(gray), model, index=batch_labels))
print("Increace in Confidence:", increase_in_confidence_list([transform(b).unsqueeze(0) for b in batch], np.array(gray), model, batch_labels))
print("Sparsity", sparsity(cam=gray))

In [ ]:
for img in minibatch:
    res, gray_res = layer_cam_gen(model, (img/255).astype(np.float32), target_layers=[model.avgpool])
    f, axarr = plt.subplots(1, 2)
    axarr[0].imshow(img)
    axarr[1].imshow(res)
    #plt.imshow(torch.movedim(torchvision.io.read_image(path + file), 0, -1).numpy())
    #plt.imshow(res)
    plt.show()

In [ ]:
for img in minibatch:
    #img = transforms(torchvision.io.read_image(path + file)).float()
    transform = torchvision.transforms.ToTensor()
    tensor = transform(img).to(device)
    res = saliency_gen(tensor, model)
    f, axarr = plt.subplots(1, 2)
    axarr[0].imshow(img)
    axarr[1].imshow(res.cpu())
    plt.show()

Deletion Game

In [ ]:
transform = torchvision.transforms.ToTensor()
diffs, percs = [], []
for img, g in zip(batch, gray):
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = deletion_game(img.to(device), cam=g, model=model, gray_res=g)
    diffs.append(diff_), percs.append(perc)

In [ ]:
np.mean(diffs), np.mean(perc)

Preservation Game

In [ ]:
diffs_preservation, percs_preservation = [], []
for img in batch:
    img = transform(img).unsqueeze(0)
    diff_, perc, cam_ = preservation_game(img.to(device), cam=g, model=model, cam_=g)
    diffs_preservation.append(diff_), percs_preservation.append(perc)

In [ ]:
np.mean(diffs_preservation), np.mean(percs_preservation)

Average Drop

In [ ]:
avg_drop_list([transform(b).unsqueeze(0) for b in batch], np.array(gray), model, index=batch_labels)

Increace in Confidence

In [ ]:
increase_in_confidence_list([transform(b).unsqueeze(0) for b in batch], np.array(gray), model, batch_labels)

Sparsity

In [ ]:
print(f" Sparsity = {sparsity(cam=gray)}")